In [0]:
import pandas as pd
import tensorflow as tf
import os
import re
import numpy as np
from string import punctuation
from zipfile import ZipFile
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
PATH='/content/gdrive/My Drive/ML Project/train.csv'

In [4]:
data = pd.read_csv(PATH)
print('총 샘플의 수 :',len(data))
train_data=data.copy()
train_data['title'].nunique(), data['text'].nunique() #기사제목, 기사내용 중복 내용 확인
# 윈도우 바탕화면에서 작업한 저자의 경우에는
# data = pd.read_csv(r'C:\Users\USER\Desktop\spam.csv',encoding='latin1')
train_data.drop_duplicates(subset=['text'], inplace=True) # v2 열에서 중복인 내용이 있다면 중복 제거
print('중복 제거 후 샘플의 수 :',len(train_data))

총 샘플의 수 : 20800
중복 제거 후 샘플의 수 : 20387


In [0]:
# importing neural network libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, GRU, LSTM, SimpleRNN, SpatialDropout1D
from tensorflow.keras.models import load_model

In [0]:
train_data = train_data.set_index('id', drop = True)

In [7]:
print(train_data.shape)
train_data.head()

(20387, 4)


,title,author,text,label
id,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
train_data.isnull().sum()

title      518
author    1854
text         1
label        0
dtype: int64

In [9]:
train_data[['title', 'author']] = train_data[['title', 'author']].fillna(value = 'Missing')
train_data = train_data.dropna()
train_data.isnull().sum()

title     0
author    0
text      0
label     0
dtype: int64

In [10]:
length = []
[length.append(len(str(text))) for text in train_data['text']]
train_data['length'] = length
train_data.head()

,title,author,text,label,length
id,,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,4930
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,4160
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,7692
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,3237
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,938


In [11]:
train_data = train_data.drop(train_data['text'][train_data['length'] < 50].index, axis = 0)
print('기사의 최대길이 : %d' % max(j for j in train_data['length']))

기사의 최대길이 : 142961


In [0]:
max_features = 4500

In [0]:
tokenizer = Tokenizer(num_words = max_features, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower = True, split = ' ')
tokenizer.fit_on_texts(texts = train_data['text'])
X = tokenizer.texts_to_sequences(texts = train_data['text'])

In [14]:
X = pad_sequences(sequences = X, maxlen = max_features, padding = 'pre')
print(X.shape)

(20263, 4500)


In [15]:
print(X.shape)
y = train_data['label'].values
print(y.shape)
print(X)
m=len(X)

(20263, 4500)
(20263,)
[[   0    0    0 ... 1096    5 4201]
 [   0    0    0 ... 3733  139 1166]
 [   0    0    0 ...    3  207  671]
 ...
 [   0    0    0 ... 3741  778   11]
 [   0    0    0 ...   10   59 3805]
 [   0    0    0 ... 1565 2266 3643]]


In [16]:
n_of_train = int(len(X) * 0.8)
n_of_test = int(len(X) - n_of_train)
print('훈련데이터 수: '+ str(n_of_train))
print('검증데이터 수: '+ str(n_of_test))
# train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=1)
print(X_valid.shape)
print(y_valid.shape)
print(X_train.shape)
print(y_train.shape)

훈련데이터 수: 16210
검증데이터 수: 4053
(4053, 4500)
(4053,)
(16210, 4500)
(16210,)


In [0]:
model = Sequential()
model.add(Embedding(4500, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

In [0]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('/content/gdrive/My Drive/ML Project/RNNbest_model2.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [19]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=30, batch_size=10, callbacks=[es, mc], validation_split=0.2)

Epoch 1/30
1297/1297 [==============================] - ETA: 0s - loss: 0.3886 - acc: 0.8253
Epoch 00001: val_acc improved from -inf to 0.90932, saving model to /content/gdrive/My Drive/ML Project/RNNbest_model2.h5
1297/1297 [==============================] - 4075s 3s/step - loss: 0.3886 - acc: 0.8253 - val_loss: 0.2505 - val_acc: 0.9093
Epoch 2/30
1297/1297 [==============================] - ETA: 0s - loss: 0.2178 - acc: 0.9163
Epoch 00002: val_acc did not improve from 0.90932
1297/1297 [==============================] - 4126s 3s/step - loss: 0.2178 - acc: 0.9163 - val_loss: 0.3534 - val_acc: 0.8371
Epoch 3/30
1297/1297 [==============================] - ETA: 0s - loss: 0.1605 - acc: 0.9417
Epoch 00003: val_acc improved from 0.90932 to 0.91672, saving model to /content/gdrive/My Drive/ML Project/RNNbest_model2.h5
1297/1297 [==============================] - 3974s 3s/step - loss: 0.1605 - acc: 0.9417 - val_loss: 0.2319 - val_acc: 0.9167
Epoch 4/30
1297/1297 [===========================

KeyboardInterrupt: ignored